# Creatig a object with the yolo model transformed into a MobileNet SSD

In [126]:
import cv2
import cvzone
import math
from itertools import combinations

import numpy as np
import os
import matplotlib.pyplot as plt
from cvzone.FaceMeshModule import FaceMeshDetector
import imutils

# Paths for MobileNet trained on caffe for person detection
prototxtPath = os.path.sep.join(["person_detector", "SSD_MobileNet_prototxt.txt"])
weightsPath = os.path.sep.join(["person_detector",
                                "SSD_MobileNet.caffemodel"])

def euclidean_dist(p1, p2):
    """
    p1, p2 = two points for calculating Euclidean Distance

    :return:
    dst = Euclidean Distance between two 2d points
    """
    if type(p1) is int or type(p1) is int:
        dst = math.sqrt(p1 ** 2 + p2 ** 2)
    else:
        dst = math.sqrt((p2[0] - p1[0] ) ** 2 + (p2[1] - p1[1])** 2)
    
    return dst

class Detector():
    
    def __init__(self):
        # Initialize our Face LandMark detector
        self.faceLandMarkDetector = FaceMeshDetector()
        
        # Initialize model for person detection
        self.net_person = cv2.dnn.readNet(prototxtPath, weightsPath)
        
        # Detected Face Land Marks
        self.detected_faces = list()
        self.red_zone_list = []  # List containing which Object id is in under threshold distance condition.
        self.red_line_list = []
        self.centroid_dict = dict()
        
        # Distances from faces to camera
        self.distances = list()
        
    def detect_distance_from_camera(self, img):
        #detect the faceMesh
        img, self.detected_faces = self.faceLandMarkDetector.findFaceMesh(img, draw=False)
    
        self.distances = list()
    
        if len(self.detected_faces) > 0:
            for face in self.detected_faces:
                pointLeft = face[145]
                pointRight = face[374]

                color = (0, 200, 0)
                #cv2.line(img, pointLeft, pointRight, color, 3)
                #cv2.circle(img, pointLeft, 5, (255,0,255), cv2.FILLED)
                #cv2.circle(img, pointRight, 5, (255,0,255), cv2.FILLED)

                w = self.faceLandMarkDetector.findDistance(pointLeft, pointRight)[0]
                #cv2.putText(img, str(w[0]), (pointLeft), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)

                # Trying to find the deep of the focal lent
                W = 6.3 #avergae distance between the eyes
                #d = 30
                #f = (w*d)/W
                f = 540
                #cv2.putText(img, str(f), (pointLeft),cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,0), 2)

                # Finding the distance from the focal point
                d = (W*f)/w

                # Create a list with the coordinates of the eyes and the distance
                self.distances.append([d, pointLeft, pointRight])
                #cv2.putText(img, "{:.2f} cm".format(d), (face[10]),cv2.FONT_HERSHEY_SIMPLEX, 2/int(d*0.047), (0,0,0), 2)

        return self.distances
    
    def detect_sd(self, frame, distances):
        frame = imutils.resize(frame, width=400)
        (h, w) = frame.shape[:2]

        #Converting Frame to Blob
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 
            0.007843, (300, 300), 127.5)

        #Passing Blob through network to detect and predict
        self.net_person.setInput(blob)
        detections = self.net_person.forward()
        detections = np.array([detection for detection in detections[0][0] if sum(detection) !=0])

        if len(detections) > 0:  # At least 1 detection in the image and check detection presence in a frame
            self.centroid_dict = dict()  # Function creates a dictionary and calls it centroid_dict
            distances_dict = dict()
            objectId = 0  # We inialize a variable called ObjectId and set it to 0

            for detection in detections:  # In this if statement, we filter all the detections for persons only
                # Check for the only person name tag 
                #name_tag = detection['name']  # Coco file has string of all the names
                #if name_tag == 'person':
                if detection[1] == 15:

                    #Extracting the confidence of predictions
                    confidence = detection[2]

                    #Filtering out weak predictions
                    if confidence > 0.7:

                        #Computing the (x,y) - coordinates of the bounding box  

                        #Extracting bounding box coordinates
                        box = detection[3:7] * np.array([w, h, w, h])
                        (xmin, ymin, xmax, ymax) = box.astype("int")

                        #Drawing the prediction and bounding box
                        label = "{}: {:.2f}%".format('person', confidence * 100)

                        y = ymin - 15 if ymin - 15 > 15 else ymin + 15
                        
                        #Store the center points of the detections
                        x = int(xmin + (xmax - xmin) / 2)
                        y = int(ymin + (ymax - ymin) / 2)
                        self.centroid_dict[objectId] = [x, y, int(xmin), int(ymin), int(xmax),
                                                   int(ymax)]  # Create dictionary of tuple with 'objectId' as the index center points and bbox
                        #[for distance in distances]
                        distances_dict[objectId] = [distance for distance in distances if xmin < distance[1][0] < xmax and ymin < distance[1][0] < ymax]
                        objectId += 1  # Increment the index for each detection


            # Determine which person are close to each other
            print('Eye distances: ', distances_dict)
            print('Persons recogized: ', self.centroid_dict)
            self.red_zone_list = []  # List containing which Object id is in under threshold distance condition.
            self.red_line_list = []
            for (id1, p1), (id2, p2) in combinations(self.centroid_dict.items(),2):  # Get all the combinations of close detections, #List of multiple items - id1 1, points 2, 1,3
                dx, dy = p1[0] - p2[0], p1[1] - p2[1]  # Check the difference between centroid x: 0, y :1
                
                #socialdist_px = euclidean_dist(p1[0:2], p2[0:2])  # Calculates the Euclidean distance

                socialdist_px = euclidean_dist(dx, dy)
                socialdist_estimated = 0
                
                distance_bepx1 = 0
                distance_bepx2 = 0
                no_eye_ref = False

                distance_fc1 = 0
                distance_fc2 = 0

                if id1 in distances_dict:
                    if len(distances_dict[id1]) > 0:
                        distance_bepx1 = euclidean_dist(distances_dict[id1][0][1], distances_dict[id1][0][2])
                        distance_fc1 = distances_dict[id1][0][0]
                if id2 in distances_dict:
                    if len(distances_dict[id2]) > 0:
                        distance_bepx2 = euclidean_dist(distances_dict[id2][0][1], distances_dict[id2][0][2])
                        distance_fc1 = distances_dict[id2][0][0]

                if distance_bepx1 > distance_bepx2:
                    socialdist = socialdist_px*6.3/distance_bepx1
                    socialdist_estimated = math.sqrt((distance_fc1-distance_fc2)**2 + (socialdist)**2)
                elif distance_bepx1 < distance_bepx2:
                    socialdist = socialdist_px*6.3/distance_bepx2
                    socialdist_estimated = math.sqrt((distance_fc2-distance_fc1)**2 + (socialdist)**2)
                else:
                    socialdist_estimated = socialdist_px
                    no_eye_ref = True

                self.centroid_dict[id1].append(socialdist_estimated)
                self.centroid_dict[id1].append(socialdist_px)
                
                self.centroid_dict[id2].append(socialdist_estimated)
                self.centroid_dict[id2].append(socialdist_px)
                
                ### Referencing horizontal distances to between eyes distance (6.3cm)
                if (socialdist_estimated < 150 and not no_eye_ref) or (socialdist_estimated < 150 and no_eye_ref):  # Set our social distance threshold - If they meet this condition then..
                    if id1 not in self.red_zone_list:
                        self.red_zone_list.append(id1)  # Add Id to a list
                        self.red_line_list.append(p1[0:2])  # Add points to the list
                        #red_zone_dists.append(socialdist)
                    if id2 not in self.red_zone_list:
                        self.red_zone_list.append(id2)  # Same for the second id
                        self.red_line_list.append(p2[0:2])
        return frame

    def draw_detections(self, img):
        # Draw detected eyes
        for distance in self.distances:
            color = (0, 200, 0)
            
            cv2.line(img, distance[1], distance[2], color, 3)
            cv2.circle(img, distance[1], 5, (255,0,255), cv2.FILLED)
            cv2.circle(img, distance[2], 5, (255,0,255), cv2.FILLED)
            cv2.putText(img, "{:.2f} cm".format(distance[0]), distance[1] ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        
        img = imutils.resize(frame, width=400)
        
        # Draw distance boxes and lines 
        for idx, box in self.centroid_dict.items():  # dict (1(key):red(value), 2 blue)  idx - key  box - value
            if idx in self.red_zone_list:  # if id is in red zone list
                #cv2.putText(frame, "{:.2f} cm".format(socialdist),(box[2], box[3]),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 0, 255),
                              2)  # Create Red bounding boxes  #starting point, ending point size of 2
            else:
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 255, 0),
                              2)  # Create Green bounding boxe bounding boxes  #starting
            if len(self.centroid_dict) > 1:
                cv2.putText(img, "Distance from person (estimated in cm): {:.2f} cm".format(box[6]), (box[2], box[3]), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)  # Display Text
                cv2.putText(img, "Distance from person (pixels): {:.2f} cm".format(box[7]), (box[2], box[3]-8), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(img, "Person {}".format(idx), (box[2], box[3]-16), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
        # Show how many people there are on risk
        text = "People at Risk: %s" % str(len(self.red_zone_list))  # Count People at Risk
        location = (10, 25)  # Set the location of the displayed text
        cv2.putText(img, text, location, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1,
                    cv2.LINE_AA)  # Display Text

        for i in range(0,len(self.red_line_list)-1):
            if i%2 == 0:
                cv2.line(img, self.red_line_list[i],self.red_line_list[i+1], (0, 0, 255), 1)
                #cv2.putText(frame, "{:.2f} cm".format(socialdist),(),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
                
        return img
    
    

In [127]:
import cv2
import cvzone
import numpy as np
import io
import traceback

cap = cv2.VideoCapture(0)
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)

detector = Detector()

try:
    while True:
        success, frame = cap.read()
        
        distances = detector.detect_distance_from_camera(frame)
        detector.detect_sd(frame, distances)
        """
        frame = detect_sd(frame, distances)
        
        #######################################
        #    Face Mask Detection
        ########################################
        
        # Draw the detected objects
        detected_frame = detect(frame)
        detected_frame = cv2.cvtColor(detected_frame, cv2.COLOR_RGB2BGR)
        
        ########################################
        # Show on screen
        ########################################
        """
        
        detected_frame = detector.draw_detections(frame)
        cv2.resizeWindow('Image', 600,600)
        cv2.imshow("Image", detected_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            break
except Exception as errors:
    errors = io.StringIO()
    traceback.print_exc(file=errors)
    contents = str(errors.getvalue())
    print(contents)
    errors.close()
finally:
    cv2.destroyAllWindows()
    cap.release()

Eye distances:  {0: []}
Persons recogized:  {0: [226, 246, 130, 192, 323, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [226, 246, 131, 192, 321, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [228, 246, 134, 192, 322, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [227, 246, 133, 191, 322, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [226, 246, 130, 192, 322, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [227, 246, 132, 192, 323, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [226, 246, 131, 192, 322, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [227, 246, 132, 191, 323, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [227, 245, 133, 190, 322, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [228, 246, 135, 191, 322, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [227, 246, 134, 191, 321, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [227, 244, 132, 188, 322, 301]}
Eye distances:  {0: []}
Persons recogize

Eye distances:  {0: []}
Persons recogized:  {0: [301, 217, 255, 167, 347, 268]}
Eye distances:  {0: [], 1: [[35.73931662604968, [102, 72], [192, 103]]]}
Persons recogized:  {0: [303, 198, 259, 148, 347, 248], 1: [93, 91, 0, -1, 187, 184]}
Eye distances:  {0: [[34.35662623782625, [99, 47], [192, 81]]], 1: []}
Persons recogized:  {0: [183, 150, 0, 1, 366, 300], 1: [303, 179, 259, 129, 347, 229]}
Eye distances:  {0: [[33.936956062158146, [102, 45], [197, 77]]], 1: [], 2: []}
Persons recogized:  {0: [185, 150, 0, 1, 371, 300], 1: [303, 175, 256, 125, 351, 225], 2: [243, 179, 209, 128, 277, 230]}
Eye distances:  {0: [[34.4746149476043, [109, 55], [202, 88]]], 1: [], 2: [[34.4746149476043, [109, 55], [202, 88]]]}
Persons recogized:  {0: [184, 150, 1, 1, 367, 300], 1: [305, 177, 259, 126, 351, 228], 2: [97, 92, -1, 1, 196, 183]}
Eye distances:  {0: [[35.73931662604968, [117, 67], [207, 98]]], 1: []}
Persons recogized:  {0: [178, 150, 0, 0, 357, 301], 1: [301, 179, 255, 129, 348, 230]}
Eye dis

Eye distances:  {0: [[55.007079758240316, [195, 220], [255, 235]]], 1: [], 2: []}
Persons recogized:  {0: [115, 160, 30, 59, 201, 262], 1: [281, 255, 247, 213, 315, 297], 2: [234, 254, 208, 212, 261, 296]}
Eye distances:  {0: [], 1: [[55.007079758240316, [197, 221], [257, 236]]]}
Persons recogized:  {0: [281, 255, 246, 213, 316, 298], 1: [117, 160, 32, 59, 202, 262]}
Eye distances:  {0: [[53.94566036359523, [198, 221], [259, 237]]], 1: [], 2: []}
Persons recogized:  {0: [123, 177, 6, 58, 241, 296], 1: [281, 255, 247, 214, 316, 297], 2: [236, 255, 210, 214, 262, 296]}
Eye distances:  {0: [], 1: [[55.65097279785506, [201, 222], [260, 238]]]}
Persons recogized:  {0: [282, 255, 248, 214, 316, 297], 1: [124, 176, 5, 57, 244, 296]}
Eye distances:  {0: [], 1: [[55.88324309797462, [203, 223], [262, 238]]]}
Persons recogized:  {0: [282, 256, 248, 214, 317, 298], 1: [128, 176, 1, 56, 255, 296]}
Eye distances:  {0: [[55.88324309797462, [204, 223], [263, 238]]], 1: []}
Persons recogized:  {0: [128

Eye distances:  {0: [[58.40397137763018, [251, 227], [308, 239]]], 1: []}
Persons recogized:  {0: [169, 186, 3, 72, 335, 300], 1: [330, 259, 291, 218, 370, 300]}
Eye distances:  {0: [[58.40397137763018, [254, 227], [311, 239]]]}
Persons recogized:  {0: [158, 184, 2, 68, 315, 301]}
Eye distances:  {0: [[59.610865846656054, [254, 227], [310, 238]]], 1: []}
Persons recogized:  {0: [162, 183, 2, 66, 322, 301], 1: [287, 255, 256, 213, 318, 297]}
Eye distances:  {0: [[58.60293386483961, [253, 227], [310, 238]]], 1: [], 2: []}
Persons recogized:  {0: [163, 184, 2, 67, 325, 301], 1: [348, 257, 307, 216, 389, 299], 2: [288, 255, 256, 213, 320, 298]}
Eye distances:  {0: [[58.40397137763018, [253, 227], [310, 239]]], 1: [], 2: []}
Persons recogized:  {0: [155, 184, 2, 68, 308, 301], 1: [348, 258, 306, 217, 390, 299], 2: [288, 256, 255, 213, 321, 300]}
Eye distances:  {0: [[57.62791647745284, [252, 227], [310, 238]]], 1: [], 2: []}
Persons recogized:  {0: [159, 184, 3, 68, 315, 300], 1: [350, 260,

Eye distances:  {0: [], 1: []}
Persons recogized:  {0: [126, 191, -2, 83, 254, 300], 1: [347, 272, 302, 247, 393, 298]}
Eye distances:  {0: [[54.33645185324489, [93, 247], [149, 275]]]}
Persons recogized:  {0: [132, 189, -1, 79, 265, 299]}
Eye distances:  {0: [[54.72169739316038, [100, 245], [156, 272]]]}
Persons recogized:  {0: [136, 189, 0, 79, 273, 299]}
Eye distances:  {0: [[55.100794790717224, [103, 241], [159, 267]]], 1: []}
Persons recogized:  {0: [136, 188, 0, 77, 272, 300], 1: [362, 271, 327, 244, 397, 299]}
Eye distances:  {0: [[54.72169739316038, [101, 238], [157, 265]]], 1: []}
Persons recogized:  {0: [137, 188, -1, 77, 275, 300], 1: [362, 271, 327, 243, 397, 299]}
Eye distances:  {0: [[54.33645185324489, [103, 244], [159, 272]]], 1: []}
Persons recogized:  {0: [137, 189, 0, 79, 274, 300], 1: [363, 272, 329, 245, 397, 299]}
Eye distances:  {0: [[54.72169739316038, [106, 247], [162, 274]]], 1: []}
Persons recogized:  {0: [137, 189, 0, 79, 274, 300], 1: [363, 273, 329, 246, 3

...

Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogiz

Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogiz

Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogiz

Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogiz

Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogized:  {}
Eye distances:  {}
Persons recogiz